# Text Features and Embeddings In CatBoost

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/events/2020_11_18_catboost_tutorial/text_embedding_features.ipynb)


**Set GPU as hardware accelerator**

First of all, you need to select GPU as hardware accelerator. There are two simple steps to do so:
Step 1. Navigate to **Runtime** menu and select **Change runtime type**
Step 2. Choose **GPU** as hardware accelerator.
That's all!

Let's install CatBoost.

In [ ]:
!pip install catboost

In [ ]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import catboost
print(catboost.__version__)

## Preparing data

In this tutorial we will use dataset **IMDB** from [Kaggle](https://www.kaggle.com) competition for our experiments. Data can be downloaded [here](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews).

In [ ]:
!wget https://transfersh.com/ou7jB/imdb.csv -O imdb.csv
df = pd.read_csv('imdb.csv')
df['label'] = (df['sentiment'] == 'positive').astype(int)
df.drop(['sentiment'], axis=1, inplace=True)
df.head()

In [ ]:
from catboost import Pool
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, train_size=0.8, random_state=0)
y_train, X_train = train_df['label'], train_df.drop(['label'], axis=1)
y_test, X_test = test_df['label'], test_df.drop(['label'], axis=1)

train_pool = Pool(data=X_train, label=y_train, text_features=['review'])
test_pool = Pool(data=X_test, label=y_test, text_features=['review'])

print('Train dataset shape: {}\n'.format(train_pool.shape))

In [ ]:
from catboost import CatBoostClassifier

def fit_model(train_pool, test_pool, **kwargs):
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.05,
        eval_metric='AUC',
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=test_pool,
        verbose=100,
    )

model = fit_model(train_pool, test_pool, task_type='GPU')

## How it works?

1. **Text Tokenization**
2. **Dictionary Creation**
3. **Feature Calculation**

## Text Tokenization

Usually we get our text as a sequence of Unicode symbols. So, if the task isn't a DNA classification we don't need such granularity, moreover, we need to extract more complicated entities, e.g. words. The process of extraction tokens -- words, numbers, punctuation symbols or special symbols which defines emoji from a sequence is called **tokenization**.<br>

Tokenization is the first part of text preprocessing in CatBoost and performed as a simple splitting a sequence on a string pattern (e.g. space).

In [ ]:
text_small = [
    "Cats are so cute :)",
    "Mouse scare...",
    "The cat defeated the mouse",
    "Cute: Mice gather an army!",
    "Army of mice defeated the cat :(",
    "Cat offers peace",
    "Cat is scared :(",
    "Cat and mouse live in peace :)"
]

target_small = [1, 0, 1, 1, 0, 1, 0, 1]

In [ ]:
from catboost.text_processing import Tokenizer

simple_tokenizer = Tokenizer()

def tokenize_texts(texts):
    return [simple_tokenizer.tokenize(text) for text in texts]

simple_tokenized_text = tokenize_texts(text_small)
simple_tokenized_text

### More preprocessing!

Lets take a closer look on the tokenization result of small text example -- the tokens contains a lot of mistakes:

1. They are glued with punctuation 'Cute:', 'army!', 'skare...'.
2. The words 'Cat' and 'cat', 'Mice' and 'mice' seems to have same meaning, perhaps they should be the same tokens.
3. The same problem with tokens 'are'/'is' -- they are inflected forms of same token 'be'.

**Punctuation handling**, **lowercasing**, and **lemmatization** processes help to overcome these problems.

### Punctuation handling and lowercasing

In [ ]:
tokenizer = Tokenizer(
    lowercasing=True,
    separator_type='BySense',
    token_types=['Word', 'Number']
)

tokenized_text = [tokenizer.tokenize(text) for text in text_small]
tokenized_text

### Removing stop words

**Stop words** - the words that are considered to be uninformative in this task, e.g. function words such as *the, is, at, which, on*.
Usually stop words are removed during text preprocessing to reduce the amount of information that is considered for further algorithms.
Stop words are collected manually (in dictionary form) or automatically, for example taking the most frequent words.

In [ ]:
stop_words = set(('be', 'is', 'are', 'the', 'an', 'of', 'and', 'in'))

def filter_stop_words(tokens):
    return list(filter(lambda x: x not in stop_words, tokens))
    
tokenized_text_no_stop = [filter_stop_words(tokens) for tokens in tokenized_text]
tokenized_text_no_stop

### Lemmatization

Lemma (Wikipedia) -- is the canonical form, dictionary form, or citation form of a set of words.<br>
For example, the lemma "go" represents the inflected forms "go", "goes", "going", "went", and "gone".<br>
The process of convertation word to its lemma called **lemmatization**.


In [ ]:
import nltk

nltk_data_path = os.path.join(os.path.dirname(nltk.__file__), 'nltk_data')
nltk.data.path.append(nltk_data_path)
nltk.download('wordnet', nltk_data_path)

lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_tokens_nltk(tokens):
    return list(map(lambda t: lemmatizer.lemmatize(t), tokens))

In [ ]:
text_small_lemmatized_nltk = [lemmatize_tokens_nltk(tokens) for tokens in tokenized_text_no_stop]
text_small_lemmatized_nltk

Now words with same meaning represented by the same token, tokens are not glued with punctuation.

<span style="color:red">Be carefull.</span> You should verify for your own task:<br>
Is it realy necessary to remove punctuation, lowercasing sentences or performing a lemmatization and/or by word tokenization?<br>

### Let's check up accuracy with new text preprocessing

Since CatBoost doesn't perform spacing punctuation, lowercasing letters and lemmatization, we need to preprocess text manually and then pass it to learning algorithm.

Since the natural text features is only synopsis and review, we will preprocess only them.

In [ ]:
%%time

def preprocess_data(X):
    X_preprocessed = X.copy()
    X_preprocessed['review'] = X['review'].apply(lambda x: ' '.join(lemmatize_tokens_nltk(tokenizer.tokenize(x))))
    return X_preprocessed

X_preprocessed_train = preprocess_data(X_train)
X_preprocessed_test = preprocess_data(X_test)

train_processed_pool = Pool(
    X_preprocessed_train, y_train, 
    text_features=['review'],
)

test_processed_pool = Pool(
    X_preprocessed_test, y_test, 
    text_features=['review'],
)

In [ ]:
model_on_processed_data = fit_model(train_processed_pool, test_processed_pool, task_type='GPU')

In [ ]:
def print_score_diff(first_model, second_model):
    first_accuracy = first_model.best_score_['validation']['AUC']
    second_accuracy = second_model.best_score_['validation']['AUC']

    gap = (second_accuracy - first_accuracy) / first_accuracy * 100

    print('{} vs {} ({:+.2f}%)'.format(first_accuracy, second_accuracy, gap))
    
print_score_diff(model, model_on_processed_data)

## Dictionary Creation

After the first stage, preprocessing of text and tokenization, the second stage starts. The second stage uses the prepared text to select a set of units, which will be used for building new numerical features.

A set of selected units is called dictionary. It might contain words, word bigramms, or character n-gramms.

In [ ]:
from catboost.text_processing import Dictionary

In [ ]:
text_small_lemmatized_nltk

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0, max_dictionary_size=10)

dictionary.fit(text_small_lemmatized_nltk);
#dictionary.fit(text_small, tokenizer)

In [ ]:
dictionary.save('dictionary.tsv')
!cat dictionary.tsv

In [ ]:
dictionary.apply([text_small_lemmatized_nltk[0]])

## Feature Calculation

### Convertation into fixed size vectors

The majority of classic ML algorithms are computing and performing predictions on a fixed number of features $F$.<br>
That means that learning set $X = \{x_i\}$ contains vectors $x_i = (a_0, a_1, ..., a_F)$ where $F$ is constant.

Since text object $x$ is not a fixed length vector, we need to perform preprocessing of the origin set $D$.<br>
One of the simplest text to vector encoding technique is **Bag of words (BoW)**.

### Bag of words algorithm

The algorithm takes in a dictionary and a text.<br>
During the algorithm text $x = (a_0, a_1, ..., a_k)$ converted into vector $\tilde x = (b_0, b_1, ..., b_F)$,<br> where $b_i$ is 0/1 (depending on whether there is a word with id=$i$ from dictionary into text $x$).

In [ ]:
X_proc_train_small, y_train_small = X_preprocessed_train[:1000]['review'].to_list(), y_train[:1000]
X_proc_train_small = list(map(simple_tokenizer.tokenize, X_proc_train_small))
X_proc_test_small, y_test_small = X_preprocessed_test[:1000]['review'].to_list(), y_test[:1000]
X_proc_test_small = list(map(simple_tokenizer.tokenize, X_proc_test_small))

dictionary = Dictionary(max_dictionary_size=100)
dictionary.fit(X_proc_train_small);

In [ ]:
def bag_of_words(tokenized_text, dictionary):
    features = np.zeros((len(tokenized_text), dictionary.size))
    for i, tokenized_sentence in enumerate(tokenized_text):
        indices = np.array(dictionary.apply([tokenized_sentence])[0])
        if len(indices) > 0:
            features[i, indices] = 1
    return features

X_bow_train_small = bag_of_words(X_proc_train_small, dictionary)
X_bow_test_small = bag_of_words(X_proc_test_small, dictionary)
X_bow_train_small.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
from sklearn.metrics import roc_auc_score

def fit_linear_model(X, y):
    model = LogisticRegression()
    model.fit(X, y)
    return model

def evaluate_model_auc(model, X, y):
    y_pred = model.predict_proba(X)[:,1]
    metric = roc_auc_score(y, y_pred)
    print('AUC: ' + str(metric))

In [ ]:
def evaluate_models(X_train, y_train, X_test, y_test):
    linear_model = fit_linear_model(X_train, y_train)
        
    print('Linear model')
    evaluate_model_auc(linear_model, X_test, y_test)
    print('Comparing to constant prediction')
    auc_constant_prediction = roc_auc_score(y_test, np.ones(shape=(len(y_test), 1)) * 0.5)
    print('AUC: ' + str(auc_constant_prediction))
    
evaluate_models(X_bow_train_small, y_train_small, X_bow_test_small, y_test_small)

In [ ]:
unigram_dictionary = Dictionary(occurence_lower_bound=0, max_dictionary_size=1000)
unigram_dictionary.fit(X_proc_train_small)

X_bow_train_small = bag_of_words(X_proc_train_small, unigram_dictionary)
X_bow_test_small = bag_of_words(X_proc_test_small, unigram_dictionary)
print(X_bow_train_small.shape)

evaluate_models(X_bow_train_small, y_train_small, X_bow_test_small, y_test_small)

### Looking at sequences of letters / words

Let's look at the example: texts 'The cat defeated the mouse' and 'Army of mice defeated the cat :('<br>
Simplifying it we have three tokens in each sentence 'cat defeat mouse' and 'mouse defeat cat'.<br>
After applying BoW we get two equal vectors with the opposite meaning:

| cat | mouse | defeat |
|-----|-------|--------|
| 1   | 1     | 1      |
| 1   | 1     | 1      |

How to distinguish them?
Lets add sequences of words as a single tokens into our dictionary:

| cat | mouse | defeat | cat_defeat | mouse_defeat | defeat_cat | defeat_mouse |
|-----|-------|--------|------------|--------------|------------|--------------|
| 1   | 1     | 1      | 1          | 0            | 0          | 1            |
| 1   | 1     | 1      | 0          | 1            | 1          | 0            |

**N-gram** is a continguous sequence of $n$ items from a given sample of text or speech (Wikipedia).<br>
In example above Bi-gram (Bigram) = 2-gram of words.

Ngrams help to add into vectors more information about text structure, moreover there are n-grams has no meanings in separation, for example, 'Mickey Mouse company'.

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0, gram_order=2)
dictionary.fit(text_small_lemmatized_nltk)

dictionary.save('dictionary.tsv')
!cat dictionary.tsv

In [ ]:
bigram_dictionary = Dictionary(occurence_lower_bound=0, max_dictionary_size=5000, gram_order=2)
bigram_dictionary.fit(X_proc_train_small)

X_bow_train_small = bag_of_words(X_proc_train_small, bigram_dictionary)
X_bow_test_small = bag_of_words(X_proc_test_small, bigram_dictionary)
print(X_bow_train_small.shape)

evaluate_models(X_bow_train_small, y_train_small, X_bow_test_small, y_test_small)

### Unigram + Bigram

In [ ]:
X_bow_train_small = np.concatenate((
    bag_of_words(X_proc_train_small, unigram_dictionary),
    bag_of_words(X_proc_train_small, bigram_dictionary)
), axis=1)
X_bow_test_small = np.concatenate((
    bag_of_words(X_proc_test_small, unigram_dictionary),
    bag_of_words(X_proc_test_small, bigram_dictionary)
), axis=1)
print(X_bow_train_small.shape)

evaluate_models(X_bow_train_small, y_train_small, X_bow_test_small, y_test_small)

## CatBoost Configuration

Parameter names:

1. **Text Tokenization** - `tokenizers`
2. **Dictionary Creation** - `dictionaries`
3. **Feature Calculation** - `feature_calcers`

\* More complex configuration with `text_processing` parameter

### `tokenizers`

Tokenizers used to preprocess Text type feature columns before creating the dictionary.

[Documentation](https://catboost.ai/docs/references/tokenizer_options.html).

```
tokenizers = [{
	'tokenizerId': 'Space',
	'delimiter': ' ',
	'separator_type': 'ByDelimiter',
},{
	'tokenizerId': 'Sense',
	'separator_type': 'BySense',
}]
```

### `dictionaries`

Dictionaries used to preprocess Text type feature columns.

[Documentation](https://catboost.ai/docs/references/dictionaries_options.html).

```
dictionaries = [{
	'dictionaryId': 'Unigram',
	'max_dictionary_size': '50000',
	'gram_count': '1',
},{
	'dictionaryId': 'Bigram',
	'max_dictionary_size': '50000',
	'gram_count': '2',
},{
	'dictionaryId': 'Trigram',
	'token_level_type': 'Letter',
	'max_dictionary_size': '50000',
	'gram_count': '3',
}]
```

### `feature_calcers`

Feature calcers used to calculate new features based on preprocessed Text type feature columns.

1. **`BoW`**<br>
Bag of words: 0/1 features (text sample has or not token_id).<br>
Number of produced numeric features = dictionary size.<br>
Parameters: `top_tokens_count` - maximum number of tokens that will be used for vectorization in bag of words, the most frequent $n$ tokens are taken (**highly affect both on CPU ang GPU RAM usage**).

2. **`NaiveBayes`**<br>
NaiveBayes: [Multinomial naive bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_naive_Bayes) model. As many new features as classes are added. This feature is calculated by analogy with counters in CatBoost by permutation ([estimation of CTRs](https://catboost.ai/docs/concepts/algorithm-main-stages_cat-to-numberic.html)). In other words, a random permutation is made and then we go from top to bottom on the dataset and calculate the probability of its belonging to this class for each object.

3. **`BM25`**<br>
[BM25](https://en.wikipedia.org/wiki/Okapi_BM25). As many new features as classes are added. The idea is the same as in Naive Bayes, but for each class we calculate not the conditional probability, but a certain relevance, which is similar to tf-idf, where the tokens instead of the words and the classes instead of the documents (or rather, the unification of all texts of this class). Only the tf multiplier in BM25 is replaced with another multiplier, which gives an advantage to classes that contain rare tokens.

```
feature_calcers = [
	'BoW:top_tokens_count=1000',
	'NaiveBayes',
	'BM25',
]
```

### `text_processing`

```
text_processing = {
    "tokenizers" : [{
        "tokenizer_id" : "Space",
        "separator_type" : "ByDelimiter",
        "delimiter" : " "
    }],

    "dictionaries" : [{
        "dictionary_id" : "BiGram",
        "max_dictionary_size" : "50000",
        "occurrence_lower_bound" : "3",
        "gram_order" : "2"
    }, {
        "dictionary_id" : "Word",
        "max_dictionary_size" : "50000",
        "occurrence_lower_bound" : "3",
        "gram_order" : "1"
    }],

    "feature_processing" : {
        "default" : [{
            "dictionaries_names" : ["BiGram", "Word"],
            "feature_calcers" : ["BoW"],
            "tokenizers_names" : ["Space"]
        }, {
            "dictionaries_names" : ["Word"],
            "feature_calcers" : ["NaiveBayes"],
            "tokenizers_names" : ["Space"]
        }],
    }
}
```

## Summary: Text features in CatBoost

### The algorithm:
1. Input text is loaded as a usual column. ``text_column: [string]``.
2. Each text sample is tokenized via splitting by space. ``tokenized_column: [[string]]``.
3. Dictionary estimation.
4. Each string in tokenized column is converted into token_id from dictionary. ``text: [[token_id]]``.
5. Depending on the parameters CatBoost produce features basing on the resulting text column: Bag of words, Multinomial naive bayes or Bm25.
6. Computed float features are passed into the usual CatBoost learning algorithm.

# Embeddings In CatBoost

### Get Embeddings

In [ ]:
# from sentence_transformers import SentenceTransformer
# big_model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
# X_embed_train = big_model.encode(X_train['review'].to_list())
# X_embed_test = big_model.encode(X_test['review'].to_list())

!wget https://transfersh.com/HDHxy/embedded_train.npy -O embedded_train.npy
X_embed_train = np.load('embedded_train.npy')

!wget https://transfersh.com/whOm3/embedded_test.npy -O embedded_test.npy
X_embed_test = np.load('embedded_test.npy')

### Experiments

In [ ]:
X_embed_first_train_small, y_first_train_small = X_embed_train[:5000], y_train[:5000]
X_embed_second_train_small, y_second_train_small = X_embed_train[5000:10000], y_train[5000:10000]
X_embed_test_small, y_test_small = X_embed_test[:5000], y_test[:5000]

#### Pure embeddings

In [ ]:
evaluate_models(X_embed_second_train_small, y_second_train_small, X_embed_test_small, y_test_small)

#### Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(solver='svd')
lda.fit(X_embed_first_train_small, y_first_train_small)

X_lda_train_small = lda.transform(X_embed_second_train_small)
X_lda_test_small = lda.transform(X_embed_test_small)
print(X_lda_train_small.shape)
evaluate_models(X_lda_train_small, y_second_train_small, X_lda_test_small, y_test_small)

### Embeddings in CatBoost

In [ ]:
import csv
with open('train_embed_text.tsv', 'w') as f:
    writer = csv.writer(f, delimiter='\t', quotechar='"')
    for y, text, row in zip(y_train, X_preprocessed_train['review'].to_list(), X_embed_train):
        writer.writerow((str(y), text, ';'.join(map(str, row))))

with open('test_embed_text.tsv', 'w') as f:
    writer = csv.writer(f, delimiter='\t', quotechar='"')
    for y, text, row in zip(y_test, X_preprocessed_test['review'].to_list(), X_embed_test):
        writer.writerow((str(y), text, ';'.join(map(str, row))))
        
with open('pool_text.cd', 'w') as f:
    f.write(
        '0\tLabel\n'\
        '1\tText\n'\
        '2\tNumVector'
    )

In [ ]:
from catboost import Pool
train_embed_pool = Pool('train_embed_text.tsv', column_description='pool_text.cd')
test_embed_pool = Pool('test_embed_text.tsv', column_description='pool_text.cd')

In [ ]:
model_text_embeddings = fit_model(train_embed_pool, test_embed_pool)

In [ ]:
print_score_diff(model, model_text_embeddings)

# Thanks!